# Curve Bootstrapping

The objective of this notebook is to illustrate how to bootstrap a ZC rate curve for pricing. We start from the yield curve of different tradable instruments available on the market to build Zero-Coupon Rate curves. From there we can derive:
- Forward Rates
- ZC Bond prices -> Discount Factors

### Library imports

In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

### Useful functions

In [79]:
def forecast(target_x, arr_x, arr_y):

    """Function similar to forecast function in excel -> linear prediction"""

    # Compute coefficients
    mean_x = np.mean(arr_x)
    mean_y = np.mean(arr_y)

    b = sum((arr_x - mean_x) * (arr_y - mean_y)) / sum((arr_x - mean_x)**2)
    a = mean_x - b * mean_y

    return a + b * target_x

### Data Loading

In [43]:
DCC = 365                       # Day Count Convention
date = datetime(2024, 9, 30)    # Extraction Date
df_curve = pd.read_excel(r"Data/yield_curve_20240930.xlsx")
df_curve

,Instrument Type,Tenor,Rate or Price
0,DEPOSIT,O/N,0.00580
1,DEPOSIT,1W,0.00648
2,DEPOSIT,1M,0.00679
3,DEPOSIT,3M,0.01173
4,FUTURE,ERZ2,97.75500
5,FUTURE,ERH3,97.20000
6,FUTURE,ERM3,96.95000
7,FUTURE,ERU3,96.90000
8,SWAP,2Y,0.02750
9,SWAP,3Y,0.02830


### Building The Curve 

We will extract the ZCB prices from the rates of the various tradable instruments
- From overnight to 3M: Deposit/Loans
- From 3M to 2Y: Short Term Futures
- From 2Y+: SWAP Rates

Deposit Rates

In [64]:
# Starting with deposit rates
df_deposit = df_curve[df_curve["Instrument Type"]=="DEPOSIT"].copy()
df_deposit["Start Date"] = date

# Compute the maturities of the different investments - Manually so far
df_deposit.insert(len(df_deposit.columns), "End Date", date)
df_deposit.iloc[0, -1] = datetime(2024, 10, 1)
df_deposit.iloc[1, -1] = datetime(2024, 10, 7)
df_deposit.iloc[2, -1] = datetime(2024, 10, 31)
df_deposit.iloc[3, -1] = datetime(2024, 12, 31)

# Add the coverage (investment period)
df_deposit["Coverage"] = (df_deposit["End Date"] - df_deposit["Start Date"]).apply(lambda x: x.days/365)

# Infer the discount factor
df_deposit["Discount Factor"] = df_deposit.apply(lambda x: 1/(1+x["Rate or Price"])**x["Coverage"], axis=1)

# Infer the ZC Rate (continuously compounded rate)
df_deposit["ZC Rate"] = df_deposit.apply(lambda x: -np.log(x["Discount Factor"]) / x["Coverage"], axis=1)

df_deposit

,Instrument Type,Tenor,Rate or Price,Start Date,End Date,Coverage,Discount Factor,ZC Rate
0,DEPOSIT,O/N,0.00580,2024-09-30,2024-10-01,0.002740,0.999984,0.005783
1,DEPOSIT,1W,0.00648,2024-09-30,2024-10-07,0.019178,0.999876,0.006459
2,DEPOSIT,1M,0.00679,2024-09-30,2024-10-31,0.084932,0.999425,0.006767
3,DEPOSIT,3M,0.01173,2024-09-30,2024-12-31,0.252055,0.997065,0.011662


Futures

The futures we work with are 3M futures starting at different points in time. We recall that futures are standardised products, therefore so are the delivery and start dates.

In [ ]:
df_futures = df_curve[df_curve["Instrument Type"]=="FUTURE"].copy()

# Adding the rate. By convention futures are quoted in terms of 1 - rate
df_futures["Rates"] = (100 - df_futures["Rate or Price"]) / 100

# Adding the starting dates - Manually so far
df_futures["Start Date"] = date
df_futures.iloc[0, 4] = datetime(2024, 12, 18)
df_futures.iloc[1, 4] = datetime(2025, 3, 18)
df_futures.iloc[2, 4] = datetime(2025, 6, 18)
df_futures.iloc[3, 4] = datetime(2025, 9, 18)

# Adding the maturity date of the futures -> Start Date + 3M
df_futures["End Date"] = df_futures.apply(lambda x: x["Start Date"] + relativedelta(months=3), axis=1)

# Compute the coverage related to each investment
df_futures["Coverage"] = (df_futures["End Date"] - df_futures["Start Date"]).apply(lambda x: x.days/365)

# Compute the ZC for the various maturity dates
df_futures["ZC Rate (end)"] = df_futures.apply(lambda x: -np.log())

df_futures

,Instrument Type,Tenor,Rate or Price,Rates,Start Date,End Date,Coverage
4,FUTURE,ERZ2,97.755,0.02245,2024-12-18,2025-03-18,0.246575
5,FUTURE,ERH3,97.200,0.02800,2025-03-18,2025-06-18,0.252055
6,FUTURE,ERM3,96.950,0.03050,2025-06-18,2025-09-18,0.252055
7,FUTURE,ERU3,96.900,0.03100,2025-09-18,2025-12-18,0.249315
